In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv("ml-1m/movies.dat",sep="::",header=None,engine="python",encoding="latin-1")
users = pd.read_csv("ml-1m/users.dat",sep="::",header=None,engine="python",encoding="latin-1")
ratings = pd.read_csv("ml-1m/ratings.dat",sep="::",header=None,engine="python",encoding="latin-1")

In [3]:
training_set = pd.read_csv("ml-100k/u1.base",delimiter="\t")
test_set = pd.read_csv("ml-100k/u1.test",delimiter="\t")

In [4]:
training_set = np.array(training_set,dtype='int')
test_set = np.array(test_set,dtype='int')

In [5]:
nb_users = int(max(max(training_set[: , 0]) , max(test_set[: ,0])))
nb_movies = int(max( max(training_set[: , 1]) , max(test_set[: , 1])))

print("TOTAL NUMBER OF USERS",nb_users)
print("TOTAL NUMBER OF MOVIES",nb_movies)

TOTAL NUMBER OF USERS 943
TOTAL NUMBER OF MOVIES 1682


In [6]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ..., 
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [7]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [8]:
training_set_final  = convert(training_set)
test_set_final  = convert(test_set)

In [9]:
training_set_final_tensor  = torch.FloatTensor(training_set_final)
test_set_final_tensor  = torch.FloatTensor(test_set_final)

In [10]:
Variable(training_set_final_tensor[0]).unsqueeze(0)

tensor([[ 0.,  3.,  4.,  ...,  0.,  0.,  0.]])

In [11]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [12]:
sae = SAE()

In [13]:
critirion = nn.MSELoss()

In [14]:
optimizer = optim.RMSprop(sae.parameters() , lr= 0.01 , weight_decay=0.5)

In [15]:
nb_epoch = 200
for epoch in range(1 , nb_epoch+1):
    train_loss = 0
    s = 0
    for id_user in range(nb_users):
        input = Variable(training_set_final_tensor[id_user]).unsqueeze(0)
        target = input.clone()
        if(torch.sum(target.data > 0 ) > 0):
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = critirion(output,target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


epoch: 1 loss: tensor(1.7713)
epoch: 2 loss: tensor(1.0966)
epoch: 3 loss: tensor(1.0534)
epoch: 4 loss: tensor(1.0385)
epoch: 5 loss: tensor(1.0310)
epoch: 6 loss: tensor(1.0266)
epoch: 7 loss: tensor(1.0238)
epoch: 8 loss: tensor(1.0219)
epoch: 9 loss: tensor(1.0208)
epoch: 10 loss: tensor(1.0195)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0185)
epoch: 13 loss: tensor(1.0179)
epoch: 14 loss: tensor(1.0176)
epoch: 15 loss: tensor(1.0172)
epoch: 16 loss: tensor(1.0169)
epoch: 17 loss: tensor(1.0169)
epoch: 18 loss: tensor(1.0165)
epoch: 19 loss: tensor(1.0164)
epoch: 20 loss: tensor(1.0161)
epoch: 21 loss: tensor(1.0163)
epoch: 22 loss: tensor(1.0158)
epoch: 23 loss: tensor(1.0159)
epoch: 24 loss: tensor(1.0158)
epoch: 25 loss: tensor(1.0157)
epoch: 26 loss: tensor(1.0154)
epoch: 27 loss: tensor(1.0154)
epoch: 28 loss: tensor(1.0151)
epoch: 29 loss: tensor(1.0125)
epoch: 30 loss: tensor(1.0114)
epoch: 31 loss: tensor(1.0108)
epoch: 32 loss: tensor(1.0085)
epoch: 33 loss: t

In [35]:
# Testing the SAE
test_loss = 0
s = 0.
output
input
for id_user in range(nb_users):
    input = Variable(training_set_final_tensor[id_user]).unsqueeze(0)
    target = Variable(test_set_final_tensor[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = critirion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9490)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app
